<a href="https://colab.research.google.com/github/mamantaroking/Dashboard-for-Duopharma/blob/main/scraping_scripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install selenium requests pandas numpy pdfplumber beautifulsoup4

In [1]:
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
from urllib.parse import urljoin, quote
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import re
import sys
import os
import json
import io
import pdfplumber

# **NPRA**

## Functions

In [ ]:
def replace_newlines_with_spaces(df):
    return df.replace([r'\r', r'\n'], ' ', regex=True)

In [ ]:

# function used to filter the kategori/category and status
# columns based on their dca number
def filter_dataframe(df, dca):
    df_filtered = pd.DataFrame()

    # DCA382 is separated because of the space (' ')
    # replacing newline (\n) between PRESCRIPTION- and FULL

    if dca == 382:
        if 'Kategori' in df.columns or 'Status' in df.columns:
            df_filtered = df[
                df['Kategori'] == 'PRESCRIPTION- FULL'
            ]
            df_filtered = df_filtered[
                df_filtered['Status'] == 'NEW PRODUCT APPROVED'
                ]

        '''if 'Status' in df.columns:
            df_filtered = df_filtered[
                df_filtered['Status'] == 'NEW PRODUCT APPROVED'
            ]'''

    elif dca == 353:
        if 'Status' in df.columns:
            df_filtered = df[
                df['Status'].isin(
                    [
                        'New Registration Approval',
                        'New Product Registration',
                        'NEW PRODUCT REGISTRATION',
                        'New Registration',
                        'New Product Approved',
                        'New Registrati on Approval',
                        'New Registrati on',
                        'NEW PRODUCT'
                    ]
                )
            ]

            df_filtered = df_filtered[[
                'Bil.',
                'No. Rujukan',
                'No. Pendaftaran MAL',
                'Nama Produk',
                'Pemegang',
                'Pengilang',
                'Tarikh Daftar',
                'Tarikh Luput',
                'Status',
            ]]

    # Same as DCA382
    elif dca < 363 and dca > 353:
        if 'Kategori' in df.columns or 'Status' in df.columns:
            df_filtered = df[
                df['Kategori'].isin(
                    [
                        'PRESCRIPTION- FULL',
                        'PRESCRIPTI ON-FULL',
                        'PRESCRIPTION-FULL'
                    ]
                )
            ]

            df_filtered = df_filtered[
                df_filtered['Status'] == 'NEW PRODUCT APPROVED'
            ]

        '''if 'Status' in df.columns:
            df_filtered = df_filtered[
                df_filtered['Status'] == 'NEW PRODUCT APPROVED'
            ]'''

    # Same reason to previous condition but with more variations
    # There're 4 keywords: New Registration Approval/Approved,
    # New Product Approved, New Product Registration
    #  and New Registration
    elif dca < 354 or dca == 33 and dca != 353:
        if 'Status' in df.columns:
            df_filtered = df[
                df['Status'].isin(
                    [
                        'New Registration Approval',
                        'New Product Registration',
                        'NEW PRODUCT REGISTRATION',
                        'New Registration',
                        'New Product Approved',
                        'New Registrati on Approval',
                        'New Registrati on'
                    ]
                )
            ]

        # No filtering applied if neither column is found
        else:
            print("Status is not in the DataFrame.")
            df_filtered = df

    # To filter the rest of the dca files
    else:
        if 'Kategori' in df.columns:
            df_filtered = df[
                df['Kategori'] == 'PRESCRIPTION-FULL'
            ]

        elif 'Category' in df.columns:
            df_filtered = df[
                df['Category'] == 'PRESCRIPTION-FULL'
            ]

        # No filtering applied if neither column is found
        else:
            print("Neither 'Kategori' nor 'Category' is in the DataFrame.")
            df_filtered = df

        if 'Status' in df.columns:
            df_filtered = df_filtered[
                df_filtered['Status'] == 'NEW PRODUCT APPROVED'
            ]

    return df_filtered


In [ ]:

# function that takes url, and columns to extract
# the pdf file into a pandas dataframe
def process_new_entry(url, columns, text):
    print(f"Processing new entry: {url}")
    try:
        # Get the PDF from the NPRA url into session
        response = requests.get(url)
        pdf_file = io.BytesIO(response.content)

        # Using pdfplumber open the pdf file
        #  and create the pdf dataframe
        pdf = pdfplumber.open(pdf_file)
        df = pd.DataFrame(columns=columns)

        # Determine the number of pages in the PDF
        size = len(pdf.pages)

        # Using a for loop to iterate through each page
        # of the pdf to extract the data
        # within each page into a pandas dataframe
        for i in range(size):
            # set the current page of the pdf
            page = pdf.pages[i]

            # Extract table from the current page
            tables = page.extract_table()

            if tables:
                # Insert the extracted data from the
                # current page into a new dataframe
                each_page_data = pd.DataFrame(
                    tables,
                    columns=columns
                )

                # print(f"Appending data with shape: {each_page_data.shape}")

                # Concatenate the dataframe containing extracted
                # data from the current page with pdf dataframe
                df = pd.concat(
                    [df, each_page_data],
                    ignore_index=True
                )
                df['DCA Number'] = text


        # Return pdf dataframe
        return df

    # Print exception in case an error occure
    except Exception as e:
        print(f"An error occurred: {e}")


## Scraping Scripts



### *Fully automatic – All in one*

    ### *Fully automatic  All in one*

In [ ]:
# The main function of the program
def main():
    # URL of the webpage to scrape (NPRA)
    base_url = "https://www.npra.gov.my/index.php/en/informationen/new-products-indication/new-products-approved-quest3.html"
    response = requests.get(base_url)

    # Initialize beautiful soup to grab all 'a' component
    # that contains the HREF link for the pdf files
    if response.status_code == 200:
        soup = BeautifulSoup(
            response.text, 'html.parser'
        )
        links = soup.find_all('a')
        # Define array that can store multiple dataframes
        # for concatenation at the end of the program
        dfs = []

        # Define dataframe columns for DCA354 and later
        newer_columns = [
            'Bil.',
            'No. Rujukan',
            'No. Pendaftaran MAL',
            'Nama Produk',
            'Pemegang',
            'Pengilang',
            'Status',
            'Kategori',
            'Tarikh Daftar',
            'Tarikh Luput',
        ]

        # Define dataframe columns for DCA353 and prior
        older_columns = [
            'Bil.',
            'No. Rujukan',
            'No. Pendaftaran MAL',
            'Nama Produk',
            'Pemegang',
            'Pengilang',
            'Tarikh Daftar',
            'Tarikh Luput',
            'Status',
        ]

        column_355 = [
            'Bil.',
            'No. Pendaftaran MAL',
            'Nama Produk',
            'Pemegang',
            'Pengilang',
            'Status',
            'Kategori',
            'Tarikh Daftar',
            'Tarikh Luput',
        ]

        column_353 = [
            'Bil.',
            'No. Rujukan',
            'No. Pendaftaran MAL',
            'Nama Produk',
            'Pemegang',
            'Pengilang',
            'Status',
            'Tarikh Daftar',
            'Tarikh Luput',
        ]


        # try:
        # For loop to iterate through the individual
        # 'a' components to get the href links
        for link in links:
            # Find the href arguement to get the pdf urls
            href = link.get('href')
            text = link.text.strip()

            # Match the DCA number and convert into Integer
            if href and re.match(r'^DCA\s*\d+', text):
                dca_number = int(
                    re.search(r'\d+', text).group()
                )
                print(text)

                # Piece together the base url and the
                # HREF url to get access to the pdf file
                full_url = urljoin(base_url, href)
                encoded_url = quote(
                    full_url, safe=':/?=&%'
                )
                print(encoded_url)

                # Only loop between 354 and above
                if dca_number == 353:
                    df = process_new_entry(
                        encoded_url,
                        column_353,
                        text
                    )

                elif dca_number == 355:
                    df = process_new_entry(
                        encoded_url,
                        column_355,
                        text
                    )


                elif dca_number >= 354:
                    df = process_new_entry(
                        encoded_url,
                        newer_columns,
                        text
                    )

                # only loop between 353 and below
                elif dca_number < 354:
                    df = process_new_entry(
                        encoded_url,
                        older_columns,
                        text
                    )

                if df is not None:
                    df = df.reset_index(drop=True)

                    # Apply function to replace
                    # newlines with spaces
                    df = replace_newlines_with_spaces(df)

                    # Apply function that filters the
                    # dataframes for specific keywords
                    df = filter_dataframe(df, dca_number)

                    print(f"Final DataFrame shape for {text}: {df.shape}")

                    # Add in the extracted pdf into the array
                    dfs.append(df)

        # Concatenate all pdf dataframes into one big df
        df = pd.concat(dfs, ignore_index=True)
        print("Concatenating complete!")

        # Add regs admin and country column
        df.insert(0, 'Regulatory Admin', 'NPRA')
        df.insert(1, 'Country Name', 'Malaysia')
        df['Region'] = 'South East Asia (SEA)'

        # Save csv file
        df.to_csv("Filtered_DCA.csv", index=False)
        print('Saving successful!')

        # Raise exceptions if an error occured
        '''except Exception as e:
            print(f"An error occurred: {e}")
        except IndexError as e:
            print(f"IndexError occurred: {e}")
        except ValueError as e:
            print(f"ValueError occurred: {e}")'''

    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

# Main function trigger
if __name__ == "__main__":
    main()


### *Auto only one - For latest data only*

In [ ]:

# The main function of the program
def main():
    # URL of the webpage to scrape (NPRA)
    base_url = "https://www.npra.gov.my/index.php/en/informationen/new-products-indication/new-products-approved-quest3.html"
    response = requests.get(base_url)

    # Initialize beautiful soup to grab all 'a' component
    # that contains the the HREF link for the pdf files
    if response.status_code == 200:
        soup = BeautifulSoup(
            response.text, 'html.parser'
        )
        links = soup.find_all('a')
        print(links)
        # Define array that can store multiple dataframes
        # for concatenation at the end of the program
        dfs = []

        # Define dataframe columns for DCA354 and later
        newer_columns = [
            'Bil.',
            'No. Rujukan',
            'No. Pendaftaran MAL',
            'Nama Produk',
            'Pemegang',
            'Pengilang',
            'Status',
            'Kategori',
            'Tarikh Daftar',
            'Tarikh Luput',
        ]


        try:
            # For loop to iterate through the individual
            # 'a' components to get the href links
            for link in links:
                # Find the href arguement to get the pdf urls
                href = link.get('href')
                text = link.text.strip()

                # Match the DCA number and convert into Integer
                if href and re.match(r'^DCA\s*\d+', text):
                    dca_number = int(
                        re.search(r'\d+', text).group()
                    )
                    print(text)

                    # Piece together the base url and the
                    # HREF url to get access to the pdf file
                    full_url = urljoin(base_url, href)
                    encoded_url = quote(
                        full_url, safe=':/?=&%'
                    )
                    print(encoded_url)


                    # Scrape the data
                    df = process_new_entry(
                        encoded_url,
                        newer_columns,
                        text
                    )


                    if df is not None:
                        df = df.reset_index(drop=True)

                        # Apply function to replace
                        # newlines with spaces
                        df = replace_newlines_with_spaces(df)

                        # Apply function that filters the
                        # dataframes for specific keywords
                        df = filter_dataframe(df, dca_number)

                        print(
                            f"Final DataFrame shape for {text}: {df.shape}"
                        )

                        # Add in the extracted pdf into the array
                        dfs.append(df)

                        break

            # Concatenate all pdf dataframes into one big df
            df = pd.concat(dfs, ignore_index=True)
            print("Concatenating complete!")

            # Add regs admin and country column
            df.insert(0, 'Regulatory Admin', 'HSA')
            df.insert(1, 'Country Name', 'Singapore')
            df['Region'] = 'South East Asia (SEA)'

            # Save csv file
            df.to_csv("Filtered_DCA.csv", index=False)
            print('Saving successful!')

        # Raise exceptions if an error occured
        except Exception as e:
            print(f"An error occurred: {e}")
        except IndexError as e:
            print(f"IndexError occurred: {e}")
        except ValueError as e:
            print(f"ValueError occurred: {e}")

    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

# Main function trigger
if __name__ == "__main__":
    main()


### *Manual - Only one*

In [ ]:
# Working with style(filter) (Use DCA357)
import pdfplumber
import pandas as pd

# Open pdf with pdfplumber
# pdf = pdfplumber.open("DCA357.pdf")
pdf = pdfplumber.open("DCA353.pdf")

dca = 353

columns = [
            'Bil.',
            'No. Rujukan',
            'No. Pendaftaran MAL',
            'Nama Produk',
            'Pemegang',
            'Pengilang',
            'Status',
            'Tarikh Daftar',
            'Tarikh Luput',
        ]

df = pd.DataFrame(columns=columns)

# Determine the number of pages in the PDF
size = len(pdf.pages)

# Using a for loop to iterate through each page
# of the pdf to extract the data
# within each page into a pandas dataframe
for i in range(size):
    # set the current page of the pdf
    page = pdf.pages[i]

    # Extract table from the current page
    tables = page.extract_table()

    if tables:
        # Insert the extracted data from the
        # current page into a new dataframe
        each_page_data = pd.DataFrame(
            tables,
            columns=columns
        )

        # print(f"Appending data with shape: {each_page_data.shape}")

        # Concatenate the dataframe containing extracted
        # data from the current page with pdf dataframe
        df = pd.concat(
            [df, each_page_data],
            ignore_index=True
        )
        df['DCA Number'] = dca

# print(df.head())
# print(df.columns)
# print(df.columns.levels)

# Reset the column names to a single level
# df.columns = df.columns.get_level_values(0)
df = replace_newlines_with_spaces(df)  # Apply the function here
df = filter_dataframe(df, dca)  # Filter df to only include required keyword

# Filter the dataframe
# df = df[df['Catatan'] == 'New\nRegistration\nApproval']
# df = df[df['Status'] == 'NEW PRODUCT\nAPPROVED']
df.to_csv('DCA353.csv', index=False)

## Pdfplumber Visual

In [ ]:
pdf = pdfplumber.open("DCA354.pdf")
page0 = pdf.pages[0]
vis = page0.to_image()
vis.debug_tablefinder({})

# **HSA**

For Scraping NDA

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# 1st base url configuration
url1 = "https://www.hsa.gov.sg/announcements/new-drug-approval/new-drug-approvals---"
# 2nd base url configuration
url2 = 'https://www.hsa.gov.sg/announcements/new-drug-approval/new-drug-approvals-'
# headers for beautifulsoup
headers = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

# Define columns
column_def = [
    'Product Name',
    'Active Ingredient(s)',
    'Application Type',
    'Product Registrant',
    'Approval Date',
    'Registration Number'
]


# Function to generate all the different variations
# in the urls based on the date (years and months)
def generate_url_variations(start_year, end_year):

    # List of months in whole spelling
    full_months = [
        'january',
        'february',
        'march',
        'april',
        'may',
        'june',
        'july',
        'august',
        'september',
        'october',
        'november',
        'december'
    ]

    # List of months in 3 letter spellings
    short_months = [
        'jan',
        'feb',
        'mar',
        'apr',
        'may',
        'jun',
        'jul',
        'aug',
        'sep',
        'oct',
        'nov',
        'dec'
    ]

    # List array to house all the urls
    urls = []

    # Start loop to iterate through the years
    # Start year = 2016, end yer = 2024
    for year in range(start_year, end_year + 1):

        # Start loop for 3 letter spelling months
        for month in short_months:
            # For year 2019 and earlier
            if year < 2019:
                urls.append(f"{url2}{month}-{year}")

            # For year 2019
            elif year == 2019:
                if month == 'jan' or month == 'feb' or month == 'mar' or month == 'apr':
                    urls.append(f"{url2}{month}-{year}")
                elif month == 'jul':
                    pass
                else:
                    urls.append(f"{url1}{month}-{year}")

            # For year 2020
            elif year == 2020:
                if month == 'jun' or month == 'jul':
                    pass
                else:
                    urls.append(f"{url1}{month}-{year}")

            # For year 2021
            elif year == 2021:
                if month == 'jan' or month == 'feb' or month == 'mar' or month == 'apr':
                    urls.append(f"{url1}{month}-{year}")

            # For year 2022
            elif year == 2022:
                if month == 'feb':
                    urls.append(f"{url1}{month}-{year}")

            # For year 2023
            elif year == 2023:
                if month == 'sep':
                    urls.append(f"{url1}{month}-{year}")

            else:
                break
        # End loop for short spelling months

        # Start loop for whole spelliog months
        for month in full_months:

            # For year 2019
            if year == 2019:
                if month == 'july':
                    urls.append(f"{url1}{month}-{year}")
                else:
                    pass

            # For year 2020
            elif year == 2020:
                if month == 'june' or month == 'july':
                    urls.append(f"{url1}{month}-{year}")
                else:
                    pass

            # For year 2021
            elif year == 2021:
                if month == 'january' or month == 'february' or month == 'march' or month == 'april':
                    pass
                else:
                    urls.append(f"{url1}{month}-{year}")

            # For year 2022 and later
            elif year > 2021:
                urls.append(f"{url1}{month}-{year}")

            else:
                pass

        # End for loop for whole month spellings
    # End loop for years

    return urls


# Function definition to scrape the tables
def scraping(url):

    # Get response from the url
    r = requests.get(url=url, headers=headers)

    # Initialize soup
    soup = BeautifulSoup(r.content, "html.parser")

    # Define output array
    output_list = []

    # Start loop to find all table element
    for table_element in soup.findAll('tbody'):

        # Start loop to find all rows in table element
        for row in table_element.findAll('tr'):
            columns = row.findAll('td')  # find columns

            # if there is more than one column
            if len(columns) > 1:

                # Get the text from the second column
                selected_column = columns[1].get_text(strip=True)

                # Add text into the output array
                output_list.append(selected_column)
                print(output_list)
        # Enf loop to find rows
    # End loop to find tables

    # Transpose the array so the rows become columns
    reshaped_list = [
        output_list[i:i + 6] for i in range(0, len(output_list), 6)
    ]
    df = pd.DataFrame(reshaped_list, columns=column_def)

    print(df)
    print(url)
    return df


# The main function
def main():
    # Generates the URLs that will be used for scraping
    urls = generate_url_variations(2016, 2024)

    # Array to store the returned dataframes from scraping
    all_data = []

    # Iterate through the urls list
    for url in urls:
        try:
            # Get the dataframes from scraping
            df = scraping(url)

            # Append the returned dataframe into the list
            all_data.append(df)
        except Exception as e:
            print(f"Failed to scrape {url}: {e}")

    # Concatenated all the dataframes in the list into one
    concatenated_df = pd.concat(
        all_data,
        ignore_index=True
    )

    # Add regs admin and country column
    concatenated_df.insert(0, 'Regulatory Admin', 'HSA')
    concatenated_df.insert(1, 'Country Name', 'Singapore')
    concatenated_df['Region'] = 'South East Asia (SEA)'

    # Save to csv
    concatenated_df.to_csv(
        'hsa_combined_nda.csv',
        index=False
    )

# Main function trigger
if __name__ == '__main__':
    main()


Streaming output truncated to the last 5000 lines.
['VIACORAM START TABLET 3.5MG/ 2.5MGVIACORAM TABLET 7MG/ 5MG & 14MG/ 10MG', 'Amlodipine and Perindopril Arginine', 'NDA-2: New dosage form, indication, dosing regimenNDA-3: New strength', 'SERVIER (S) PTE LTD', '01/10/2018', 'SIN15555P, SIN15556P, SIN15557P', 'TARGIN PROLONGED RELEASE TABLETS 60MG/30MG AND 80MG/40MG', 'Oxycodone hydrochloride / Naloxone hydrochloride dihydrate', 'NDA-3: New strength', 'MUNDIPHARMA PHARMACEUTICALS PTE. LTD.', '02/10/2018', 'SIN15560P, SIN15561P', 'SENSHIO FILM-COATED TABLET 60MG', 'Ospemifene', 'NDA-1: New chemical entity', 'SHIONOGI SINGAPORE PTE. LTD.', '26/10/2018', 'SIN15568P', 'IMFINZI CONCENTRATE FOR SOLUTION FOR INFUSION 50MG/ML']
['VIACORAM START TABLET 3.5MG/ 2.5MGVIACORAM TABLET 7MG/ 5MG & 14MG/ 10MG', 'Amlodipine and Perindopril Arginine', 'NDA-2: New dosage form, indication, dosing regimenNDA-3: New strength', 'SERVIER (S) PTE LTD', '01/10/2018', 'SIN15555P, SIN15556P, SIN15557P', 'TARGIN PR

KeyboardInterrupt: 

For Scraping GDA

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Swap the path to web driver in the executable_path
service = Service(executable_path='C:/Users/Aiman/geckodriver-v0.35.0-win64/geckodriver.exe')
# URL path
url = 'https://www.hsa.gov.sg/therapeutic-products/approvals-and-post-reg-actions'

# Set up the Selenium WebDriver
logging.info('Setting up the WebDriver')
driver = webdriver.Firefox(service=service)

# create action chain object
action = ActionChains(driver)

# Load the website
driver.get(url)
logging.info('Navigated to the website')

# Create a DataFrame to store the data
df = pd.DataFrame(
    columns=[
    'Approval Date',
    'Product Name',
    'Active Ingredient(s)',
    'Application Type',
    'Product Registrant',
    'Registration Number'
    ]
)

# Define dictionary for the different components
gda_dict = {
    'year': [2024, 2023, 2022, 2021],

    'accordion': [
        '//*[@id="main_C187_Col00"]',
        '//*[@id="main_C166_Col00"]',
        '//*[@id="main_C123_Col00"]',
        '//*[@id="main_C124_Col00"]'
    ],

    'button': [
        '/html/body/form/main/div/div[2]/div/div[1]/div/div[5]/div[1]/div/div[2]/div/div/div/div[3]/div[2]/span[5]',
        '/html/body/form/main/div/div[2]/div/div[1]/div/div[5]/div[2]/div/div[2]/div/div/div/div/div/div[3]/div[2]/span[6]',
        '/html/body/form/main/div/div[2]/div/div[1]/div/div[5]/div[3]/div/div[2]/div/div/div/div/div/div[3]/div[2]/span[5]',
        '/html/body/form/main/div/div[2]/div/div[1]/div/div[5]/div[4]/div/div[2]/div/div/div/div/div/div[3]/div[2]/span[7]'
    ]
}

# Define index for the years
u = 0

# Start loop — each year in gda_list from 2024 to 2021
for year in gda_dict['year']:
    print(year)
    time.sleep(1)

    # Move cursor to the accordian
    logging.info('Located Accordian button')
    menu_trigger = driver.find_element(
        By.XPATH,
        f'//*[@id="togglepanel-{year}"]'
    )

    # Scrolling the accordian into view
    time.sleep(1)
    logging.info('Scrolling the Accordian element into view')
    driver.execute_script(
        "arguments[0].scrollIntoView();",
        menu_trigger
    )

    # Click the Accordian
    time.sleep(1)
    logging.info('Clicking the accordian')
    ActionChains(driver).click(
        on_element=menu_trigger
    ).perform()

    # Wait for the accordian to be loaded
    logging.info('Waiting for the Accordian content to be loaded')
    acn = gda_dict['accordion'][u]
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((
            By.XPATH,
            f'{acn}'
        ))
    )
    logging.info('Accordian loaded\n')

    print(u)

    # initialize index for pages
    i=0

    # Assign the maximum pages according to the year
    if year == 2024:
        v=3
    elif year == 2023:
        v=4
    elif year == 2022:
        v=3
    elif year == 2021:
        v=5

    # While page number is below page max
    while i < v:
        time.sleep(1)
        print(f'Page: {i}')

        # Find the accordian component
        list_element = driver.find_element(
            By.XPATH,
            f'{acn}'
        )

        # Get the html content from the accordian
        list_html = list_element.get_attribute('outerHTML')

        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(list_html, "html.parser")

        # Find the table
        div = soup.find(
            'table',
            attrs={
                'class' :
                "hsa-datatable status--done dataTable-table"
            }
        )

        time.sleep(5)

        # Array to store data from each row
        row_data = []

        # Start loop — find each column
        for idx, cols in enumerate(div.find_all('td')):

            # print(idx)

            # Grab the text in the column
            selected_column = cols.get_text(strip=True)

            # Append the text into row_data
            row_data.append(selected_column)

            # Since there are 6 columns for every row
            # when len(row_data) is 6
            # it will be dataframed as one row
            if len(row_data) == 6:
                row_df = pd.DataFrame(
                    [row_data],
                    columns=df.columns
                )
                # print(row_df)

                # Concatenate all the row df into one table df
                df = pd.concat(
                    [df, row_df],
                    ignore_index=True,
                    sort=False
                )

                # Reset the row_data
                row_data = []
                # print(row_data)

        # End loop

        # Find next page element
        btn = gda_dict['button'][u]
        next_button = driver.find_element(
            By.XPATH,
            f'{btn}'
        )
        logging.info('Found the next page button')

        # Scroll next page element into view
        logging.info('Scrolling the header element into view')
        driver.execute_script(
            "arguments[0].scrollIntoView();",
            next_button
        )

        time.sleep(1)

        # Click on the next page button
        ActionChains(driver).click(
            on_element=next_button
        ).perform()
        logging.info('Clicked on the next page button\n')

        # Add increment to the page index
        i=i+1

        # Scrolling the accordian into view
        time.sleep(1)
        logging.info('Scrolling the Accordian element into view')
        driver.execute_script(
            "arguments[0].scrollIntoView();",
            menu_trigger
        )


    logging.info('Finished Extraction')

    # Add increment to the yeat index
    u=u+1
    print(df.info())

# Add regs admin and country column
df.insert(0, 'Regulatory Admin', 'HSA')
df.insert(1, 'Country Name', 'Singapore')
df['Region'] = 'South East Asia (SEA)'

# Save dataframe to csv
df.to_csv('all_gda.csv', index=False)


To concatenate NDA and GDA datasets

In [ ]:
df1 = pd.read_csv('hsa_combined_nda.csv')
df2 = pd.read_csv('all_gda.csv')

dfA = pd.concat([df1, df2])
dfA.to_csv('full_hsa.csv', index = False)

One-time scraping for NDA, change the URL

In [ ]:
# One-by-one manually insert link
from bs4 import BeautifulSoup
import requests
import pandas as pd

URL = "https://www.hsa.gov.sg/announcements/new-drug-approval/new-drug-approvals---october-2024"
headers = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
r = requests.get(url=URL, headers=headers)

soup = BeautifulSoup(r.content, "html.parser")
output_list = []

for table_element in soup.findAll('tbody'):
    for row in table_element.findAll('tr'):
        columns = row.findAll('td')
        if len(columns) > 1:
            selected_column = columns[1].get_text(strip=True)
            output_list.append(selected_column)


print(output_list)

# Calculate the number of columns needed
num_columns = (len(output_list) + 5) // 6
reshaped_list = [
    output_list[i:i + 6] for i in range(0, len(output_list), 6)
]
df = pd.DataFrame(reshaped_list) #.transpose()

# Add regs admin and country column
df.insert(0, 'Regulatory Admin', 'HSA')
df.insert(1, 'Country Name', 'Singapore')
df['Region'] = 'South East Asia (SEA)'

print(df)
df.to_csv('hsa_october_2024.csv', index=False)

One-time scraping for GDA (this is set to scrape 2024).

In [ ]:
# 2024

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

service = Service(executable_path='C:/Users/Aiman/geckodriver-v0.35.0-win64/geckodriver.exe')
url = 'https://www.hsa.gov.sg/therapeutic-products/approvals-and-post-reg-actions'

# Set up the Selenium WebDriver
logging.info('Setting up the WebDriver')
driver = webdriver.Firefox(service=service)

driver.get(url)
logging.info('Navigated to the website')

# create action chain object
action = ActionChains(driver)

# Create a DataFrame to store the data
df = pd.DataFrame(
    columns=[
    'Approval Date',
    'Product Name',
    'Active Ingredient(s)',
    'Application Type',
    'Product Registrant',
    'Registration Number'
    ]
)

# Move cursor to the accordian
time.sleep(1)
logging.info('Located Accordian button')

# ============= Find the accordion button/element according to year ============
menu_trigger = driver.find_element(
    By.XPATH, '//*[@id="togglepanel-2024"]'
)

# Scrolling the accordian into view
time.sleep(1)
logging.info('Scrolling the Accordian element into view')
driver.execute_script(
    "arguments[0].scrollIntoView();", menu_trigger
)

# Click the Accordian
time.sleep(1)
logging.info('Clicking the accordian')
ActionChains(driver).click(on_element=menu_trigger).perform()

# Wait for the modal to be present and the table to be loaded
logging.info('Waiting for the Accordian content to be loaded')
WebDriverWait(driver, 10).until(
    # ==================== Find the table in the accordion =====================
    EC.presence_of_element_located((
        By.XPATH, '//*[@id="main_C187_Col00"]'
    ))
)
logging.info('Accordian loaded\n')
# logging.info('Start of Row 1')

i=1

while i < 4:
    time.sleep(1)
    print(f'Page: {i}')
    # ==================== Find the table in the accordion =====================
    list_element = driver.find_element(
        By.XPATH, '//*[@id="main_C187_Col00"]'
    )
    list_html = list_element.get_attribute('outerHTML')

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(list_html, "html.parser")
    div = soup.find(
        'table',
        attrs={'class' :"hsa-datatable status--done dataTable-table"}
    )

    time.sleep(5)
    row_data = []

    for idx, cols in enumerate(div.find_all('td')):
        # print(idx)
        selected_column = cols.get_text(strip=True)
        row_data.append(selected_column)

        if len(row_data) == 6:
            row_df = pd.DataFrame(
                [row_data], columns=df.columns
            )
            # print(row_df)
            df = pd.concat(
                [df, row_df], ignore_index=True, sort=False
            )
            row_data = []
            # print(row_data)

    # ========================= Find next page button ==========================
    next_button = driver.find_element(
        By.XPATH, '/html/body/form/main/div/div[2]/div/div[1]/div/div[5]/div[1]/div/div[2]/div/div/div/div[3]/div[2]/span[5]'
    )
    logging.info('Found the next page button')

    # Scroll next page element into view
    logging.info('Scrolling the header element into view')
    driver.execute_script(
        "arguments[0].scrollIntoView();", next_button
    )

    time.sleep(1)
    # Click on the next page button
    ActionChains(driver).click(on_element=next_button).perform()
    logging.info('Clicked on the next page button\n')
    i=i+1

    # Scrolling the accordian into view
    time.sleep(1)
    logging.info('Scrolling the Accordian element into view')
    driver.execute_script(
        "arguments[0].scrollIntoView();", menu_trigger
    )

df.insert(0, 'Regulatory Admin', 'HSA')
df.insert(1, 'Country Name', 'Singapore')
df['Region'] = 'South East Asia (SEA)'

df.to_csv('gda_hsa_2024.csv')

logging.info('Finished Extraction')
print(df.info())


# **BPOM**

**This code does not work now unfortunately.**

In [3]:
# Array to store multiple dataframes
output_list = []

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
    )

# Path to Web Driver
service = Service(executable_path='C:/Users/Aiman/geckodriver-v0.35.0-win64/geckodriver.exe')

# Set up the Selenium Web Driver
logging.info('Setting up the WebDriver')
driver = webdriver.Firefox(service=service)

# Open the website in browser
driver.get("https://cekbpom.pom.go.id/obat")
logging.info('Navigated to the website')

# Create action chain object
action = ActionChains(driver)

# Create a DataFrame to store the data
df = pd.DataFrame(
    columns=[
        'Nomor Registrasi',
        'Tanggal Terbit',
        'Masa Berlaku s/d',
        'Diterbitkan Oleh (1)',
        'Diterbitkan Oleh (2)',
        'Produk',
        'Nama Produk',
        'Bentuk Sediaan',
        'Komposisi',
        'Merk',
        'Kemasan',
        'Pendaftar',
        'Diproduksi Oleh'
    ]
)

# Important index to declare the first page for scraping
page_start = 1

# Important to change when website updates the list
total_data = 28493

# Start loop
'''Only starts scraping when web driver is on the correct page
Will skip the earlier pages without scraping them
Change the if-else statements when you
want the code to start scraping from an advanced page
or when the total number of data changes'''
# It will only loop when page_start isnt 1
if page_start == 1:
    pass
else:
    # Loop will stop when "start" is False
    while start:
        # Wait until next button is loaded completely
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (
                    By.XPATH,
                '//*[@id="next"]'
                )
            )
        )

        # Find the header element
        header_element = driver.find_element(
            By.XPATH,
            '/html/body/div[2]/div/div/div[2]/div/div[1]/div/div/div[1]/div/div[3]'
        )

        # Get the outer HTML script for Beautiful Soup to parse
        header_html = header_element.get_attribute(
            'outerHTML'
        )

        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(
            header_html,
            "html.parser"
        )

        # Find the component that details the page contents
        # "1 - 10 dari 28493 Data" for example
        div = soup.find(
            'div',
            attrs={
                'class':
                "kt-inbox__pages paginggall"
            }
        ).get_text(strip=True)

        # Break the loop when web driver reaches the desired page
        logging.info(div)
        # Change the page detail below
        if div == f'{page_start} - {page_start+9} dari {total_data} Data':
            print(div)
            print('Loop Break')
            start = False
            break

        else:
            # Search for header element
            header = driver.find_element(
                By.XPATH,
                '/html/body/div[2]/div/div/div[2]/div/div[1]/div/div/div[1]/div'
            )

            # Scroll the element into view
            driver.execute_script(
                "arguments[0].scrollIntoView();",
                header
            )

            # time.sleep(2)
            # Click on the next page button
            next_button = driver.find_element(
                By.XPATH,
                '//*[@id="next"]'
            )
            ActionChains(driver).click(
                on_element=next_button
            ).perform()
    # End loop


# Start loop
# When u>=0 and u<200
# this means that it will only scrape 20 pages
while 0 < page_start < 201:

    print("Start of New Loop")
    # time.sleep(5)

    # Wait for the modal to be present
    # and the list to be loaded
    # logging.info('Waiting for the list to be loaded')
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (
                By.XPATH,
             '//*[@id="detailobat"]'
            )
        )
    )
    logging.info('Table loaded\n')
    logging.info('Start of Row 1')

    # Find the list element
    list_element = driver.find_element(
        By.XPATH,
        '//*[@id="inbox-list"]'
    )

    # Get the HTML script of the list element
    list_html = list_element.get_attribute(
        'outerHTML'
    )

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(
        list_html,
        "html.parser"
    )

    # Find the list element in HTML
    div = soup.find(
        'div',
        attrs={
            'class':'kt-inbox__items',
            'id':"inbox-list",
            'data-type':"inbox",
            'style':
            "margin-top: -0.8rem; z-index: 0;"
        }
    )

    # Index for the number of rows
    i=1

    # Start Loop
    # Find all the rows within the list
    for row in div.find_all(
        'div',
        attrs={
            'class':
            'kt-inbox__item kt-inbox__item--unread'
        }):

        time.sleep(2)

        # Wait for the element to be present
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (
                    By.XPATH,
                    f'/html/body/div[2]/div/div/div[2]/div/div[1]/div/div/div[2]/div/div[{i}]'
                )
            )
        )
        logging.info(
            'Menu trigger element loaded'
        )

        # Find the element
        time.sleep(2)
        logging.info(
            'Finding the menu trigger element'
        )
        menu_trigger = driver.find_element(
            By.XPATH,
            f'/html/body/div[2]/div/div/div[2]/div/div[1]/div/div/div[2]/div/div[{i}]'
        )

        # Scroll the element into view
        logging.info(
            'Scrolling the menu trigger element into view'
        )
        time.sleep(2)
        action.move_to_element(
            menu_trigger
        ).perform()

        # Click on the element
        time.sleep(2)
        logging.info(
            'Clicking on the menu trigger element'
        )
        ActionChains(driver).click(
            on_element=menu_trigger
        ).perform()

        # Wait for the menu popup to become visible
        time.sleep(1)
        logging.info(
            'Waiting for the menu popup to become visible'
        )
        WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located(
                (
                    By.XPATH,
                    '//*[@id="exampleModal2"]'
                )
            )
        )
        logging.info('Menu popup is visible')

        # Find the menu popup element
        # time.sleep(1)
        menu_popup = driver.find_element(
            By.XPATH,
            '//*[@id="exampleModal2"]'
        )

        # Get the HTML content of the menu popup element
        # time.sleep(1)
        menu_html = menu_popup.get_attribute('outerHTML')

        # Parse the HTML content with BeautifulSoup
        # time.sleep(1)
        soup = BeautifulSoup(menu_html, "html.parser")
        div = soup.find(
            'div',
            attrs={
                'class': 'modal fade show',
                'id': "exampleModal2"
            }
        )

        # Extract the row
        time.sleep(1)
        row_data = []

        # Start Loop
        for idx, cols in enumerate(
            div.find_all(
                'td',
                attrs={
                    'class':
                    'form-field-input'
                })):

            if idx<13:
                selected_column = cols.find(
                    'span',
                    attrs={
                        'class':
                        'stepper-number label label-primary label-inline mr-2 font-weight-bold'
                    }
                ).get_text(strip=True)
                row_data.append(selected_column)
        # End Loop

        # time.sleep(5)
        logging.info('Finished Extraction')
        # print(row_data)

        # Create temporary dataframe for the row data
        row_df = pd.DataFrame(
            [row_data],
            columns=df.columns
        )
        print(row_df)

        # Concatenate row df to encompassing df
        df = pd.concat(
            [df, row_df],
            ignore_index=True, sort=False
            )

        # Find close button element
        # time.sleep(1)
        logging.info(
            'Finding the menu close element'
        )
        close = driver.find_element(
            By.XPATH,
            '/html/body/div[2]/div/div/div[2]/div/div[3]/div/div/div[3]/button'
        )
        logging.info(
            'Clicking on the menu close element'
        )
        ActionChains(driver).click(
            on_element=close
        ).perform()

        # Wait for menu popup to close
        # time.sleep(1)
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located(
                (
                    By.XPATH,
                    '//*[@id="inbox-list"]/div[1]'
                )
            )
        )
        logging.info('Menu popup closed')
        logging.info(f'End of row {i}\n')
        # time.sleep(5)


        i+=1


        if i == 4:
            # Scroll the element into view
            # time.sleep(1)
            logging.info(
                'Scrolling the menu trigger element into view'
            )
            driver.execute_script(
                "arguments[0].scrollIntoView();",
                menu_trigger
            )


        elif i == 8:
            # time.sleep(1)
            # Scroll the element into view
            logging.info(
                'Scrolling the menu trigger element into view'
            )
            driver.execute_script(
                "arguments[0].scrollIntoView();",
                menu_trigger
            )


        elif i == 11:
            # time.sleep(1)
            logging.info(f'End of Page {u}\n')
            reenter_loop = False
            print('Exiting Loop')
            i = 1
            break
    # End Loop

    # Search for header element
    header = driver.find_element(
        By.XPATH,
        '/html/body/div[2]/div/div/div[2]/div/div[1]/div/div/div[1]/div'
    )
    logging.info('Found the header element')

    # Scroll the element into view
    logging.info(
        'Scrolling the header element into view'
    )
    driver.execute_script(
        "arguments[0].scrollIntoView();",
        header
    )

    time.sleep(3)

    # Click on the next page button
    next_button = driver.find_element(
        By.XPATH, '//*[@id="next"]'
    )
    logging.info(
        'Found the next page button'
    )
    ActionChains(driver).click(
        on_element=next_button
    ).perform()
    logging.info(
        'Clicked on the next page button\n'
    )


    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(header_html, "html.parser")
    div = soup.find(
        'div',
        attrs={'class':"kt-inbox__pages paginggall"}
    ).get_text(strip=True)

    page_start+=1
    logging.info(f'Start of Page {u}\n')
    # End Loop
# End Loop

# Add regs admin and country column
df.insert(0, 'Regulatory Admin', 'BPOM')
df.insert(1, 'Country Name', 'Indonesia')
df['Region'] = 'South East Asia (SEA)'

# Save to csv
df.to_csv('bpom_1.csv')

# Close the WebDriver
# logging.info('Closing the WebDriver')
# driver.quit()
df.head()


NoSuchDriverException: Message: Unable to obtain driver for firefox; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


# **PH FDA**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# initialize index
i=1

# initialize URL path
URL = "https://verification.fda.gov.ph/drug_productslist.php?start="+str(i)
# URL = 'https://verification.fda.gov.ph/DRUGS_NEW_APPLICATIONSlist.php?start='+str(i)
print(URL)

# Define user-agent for beautiful soup
headers = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

# Define dataframe
df = pd.DataFrame(
    columns=[
        'Registration Number',
        'Generic Name',
        'Brand Name',
        'Dosage Strength',
        'Dosage Form',
        'Pharmaceutical Category',
        'Manufacturer',
        'Country of Origin',
        'Application Type',
        'Issuance Date',
        'Expiry Date',
        ]
    )

# Array to save list of dataframes
output_list = []

# initialize row count for debugging
row_count = 0


while i <= 32881:
    # Load the website and get response
    r = requests.get(url=URL, headers=headers)

    # Initialize soup
    soup = BeautifulSoup(r.content, "html.parser")

    # Find the table to scrape
    table = soup.find(
        'table',
        attrs ={'class':'table ew-table'}
    )

    # Start loop
    # For every row in the table
    for row in table.tbody.find_all('tr'):

        # Find all data for each column
        columns = row.find_all('td')

        # Ensure the row has the expected number of columns
        if len(columns) != []:

            # Scrape the data based on the columns
            noReg = columns[2].get_text(strip=True)
            generic = columns[3].get_text(strip=True)
            brand = columns[4].get_text(strip=True)
            dosageStrength = columns[5].get_text(strip=True)
            dosageForm = columns[6].get_text(strip=True)
            category = columns[7].get_text(strip=True)
            manufacturer = columns[8].get_text(strip=True)
            country = columns[9].get_text(strip=True)
            appType = columns[10].get_text(strip=True)
            dateIssued = columns[11].get_text(strip=True)
            dateExpire = columns[12].get_text(strip=True)


            # Create a temporary DataFrame for the current row
            temp_df = pd.DataFrame(
                {
                    'Registration Number': [noReg],
                    'Generic Name': [generic],
                    'Brand Name': [brand],
                    'Dosage Strength': [dosageStrength],
                    'Dosage Form': [dosageForm],
                    'Pharmaceutical Category': [category],
                    'Manufacturer': [manufacturer],
                    'Country of Origin': [country],
                    'Application Type': [appType],
                    'Issuance Date': [dateIssued],
                    'Expiry Date': [dateExpire]
                }
            )

            # Concatenate the temporary DataFrame with
            # the main DataFrame
            df = pd.concat([df, temp_df], ignore_index=True)

            # Log the row data for debugging
            print(f"Row {row_count}: {temp_df}")
            row_count += 1
            print(URL)

    # End loop

    # Add 20 increment to url
    i+=20
    URL = "https://verification.fda.gov.ph/drug_productslist.php?start="+str(i)


# print(df.head())
# print(df.info())

# Remove duplcates from df
df.drop_duplicates()

# Copy df and read df for merging
df1 = df.copy()
df2 = pd.read_csv('drug_products.csv')

# print(df1.head())
# print(df2.head())

# Merge the two df
merged_df = pd.concat([df1, df2], ignore_index=True)
# print(merged_df.info())

# Key column for row merging
column_merge = ['Registration Number']

# Function for row merging
def merge_rows(group):
    merged = group.ffill().bfill().iloc[0]
    return merged.infer_objects(copy=False)

# Apply the function
result_df = merged_df.groupby(column_merge).apply(merge_rows).reset_index(drop=True)
# print(result_df.info())

# Filter df for only appropriate data
result_df = result_df[[
    'Registration Number',
    'Generic Name',
    'Brand Name',
    'Dosage Strength',
    'Dosage Form',
    'Pharmaceutical Category',
    'Manufacturer',
    'Country of Origin',
    'Application Type',
    'Classification',
    'Packaging',
    'Trader',
    'Importer',
    'Distributor',
    'Issuance Date',
    'Expiry Date'
]]

# Add regs admin and country column
result_df.insert(0, 'Regulatory Admin', 'PH FDA')
result_df.insert(1, 'Country Name', 'Philippines')
result_df['Region'] = 'South East Asia (SEA)'

# Save to CSV
result_df.to_csv('ph_merge.csv', index=False)
# result_df.to_csv('ph_new.csv', index=False)
